Serial Schedule Generation Scheme (SGS)

The solver iteratively schedules jobs that are ready (all predecessors completed). For each ready job, it determines the earliest feasible start time by considering:

* Precedence Constraints: A job can only start after all its predecessors have finished. In this single-machine context, current_time implicitly handles this as jobs are released sequentially.
* Resource Constraints: Implicitly handled by the single-machine assumption. Only one job can run at any given time. The current_time advances to the finish time of the scheduled job.

The algorithm maintains a current_time variable to track the machine's availability and effectively finds the next available time slot.

Heuristics Employed:

* Job Prioritization (Weighted Shortest Processing Time - WSPT Variant): When multiple jobs are ready (i.e., their in_degree is 0), the solver prioritizes jobs based on a heuristic calculated as Tardiness Penalty / Duration (job['tp'] / max(job['d'], 1)). This is a variation of the WSPT rule, aiming to complete jobs with higher tardiness penalties relative to their processing time first, thus minimizing overall tardiness. The ready_queue is sorted in descending order of this priority.
* Earliest Feasible Start Time: For a selected job, the start time is set to the current machine availability (current_time). Since it's a single machine and jobs are processed one after another, this guarantees the earliest possible start for the chosen job given the sequential nature.
* Mode Selection: The code directly uses the duration (d) and unit_penalty values (ep, tp) from the job's base definition. It does not appear to handle multiple operational modes for a single job in the way a more complex resource-constrained scheduler might. It assumes a single, fixed mode with predefined duration and penalties.

In [1]:
import json
import time
from collections import deque, defaultdict

def solve_jit_scheduling(input_data):
    start_wall_time = time.time()
    
    # 1. Dynamic Extraction (Handling your specific nested structure)
    common_due_date = input_data.get("parameters", {}).get("common_due_date", 0)
    raw_jobs = input_data.get("jobs", [])
    
    # Pre-process job data and dependencies
    jobs_dict = {}
    successors = defaultdict(list)
    in_degree = {j['id']: 0 for j in raw_jobs}
    
    for j in raw_jobs:
        jid = j['id']
        # Extract nested penalties safely
        p = j.get('penalties', {})
        jobs_dict[jid] = {
            'id': jid,
            'd': j.get('duration', 0),
            'ep': p.get('earliness_unit_penalty', 0),
            'tp': p.get('tardiness_unit_penalty', 0)
        }
        # Build DAG
        preds = j.get('predecessors', [])
        for p_id in preds:
            successors[p_id].append(jid)
            in_degree[jid] += 1

    # 2. Heuristic: Priority Rule with Topological Constraint
    # We use a Weighted Shortest Processing Time (WSPT) variant for tardiness.
    # Priority = Tardiness Penalty / Duration
    def get_priority(jid):
        job = jobs_dict[jid]
        return job['tp'] / max(job['d'], 1)

    schedule = {}
    finish_times = {}
    current_time = 0
    
    # Available jobs (no remaining predecessors)
    ready_queue = [j for j in in_degree if in_degree[j] == 0]
    
    # 3. Serial Schedule Generation Scheme
    while ready_queue:
        # Sort ready jobs by the heuristic priority (highest first)
        ready_queue.sort(key=get_priority, reverse=True)
        
        curr_id = ready_queue.pop(0)
        job = jobs_dict[curr_id]
        
        # Predecessor constraint: start cannot be earlier than max finish of predecessors
        # (Though in single machine, current_time usually covers this)
        start_time = current_time
        
        # Assign to schedule
        schedule[curr_id] = start_time
        finish_time = start_time + job['d']
        finish_times[curr_id] = finish_time
        current_time = finish_time
        
        # Update dependencies
        for succ in successors[curr_id]:
            in_degree[succ] -= 1
            if in_degree[succ] == 0:
                ready_queue.append(succ)

    # 4. Metrics Calculation
    total_penalty = 0
    for jid, f_time in finish_times.items():
        job = jobs_dict[jid]
        if f_time < common_due_date:
            total_penalty += (common_due_date - f_time) * job['ep']
        elif f_time > common_due_date:
            total_penalty += (f_time - common_due_date) * job['tp']

    return {
        "schedule": schedule,
        "metrics": {
            "makespan": current_time,
            "total_penalty": total_penalty,
            "computation_time_ms": (time.time() - start_wall_time) * 1000
        }
    }

In [2]:
with open("C:/Users/Admin/Desktop/sch50.json", 'r') as f:
    data = json.load(f)  
result = solve_jit_scheduling(data)
print(result)

{'schedule': {6: 0, 10: 0, 20: 0, 39: 0, 2: 0, 8: 0, 15: 0, 31: 0, 11: 0, 21: 0, 29: 0, 30: 0, 40: 0, 22: 0, 32: 0, 38: 0, 9: 0, 14: 0, 27: 0, 42: 0, 46: 0, 16: 0, 23: 0, 25: 0, 33: 0, 41: 0, 44: 0, 49: 0, 28: 0, 47: 0, 3: 0, 18: 0, 19: 0, 24: 0, 36: 0, 48: 0, 26: 0, 45: 0, 1: 0, 17: 0, 43: 0, 4: 0, 12: 0, 13: 0, 34: 0, 37: 0, 7: 0, 35: 0, 50: 0, 5: 0}, 'metrics': {'makespan': 0, 'total_penalty': 27392, 'computation_time_ms': 0.0}}
